In [ ]:
!pip install imagen-pytorch==1.24.2
from imagen_pytorch import Unet, SRUnet256, Imagen, ImagenTrainer
from imagen_pytorch.data import Dataset
from accelerate import notebook_launcher
import torch
import numpy as np
import pickle
import json
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files, drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
unet1 = Unet(
    dim = 192, #work on this 256 128 have been tested
    dim_mults = (1, 2, 4),
    num_resnet_blocks = 3,
    layer_attns = (False, True, True),
    layer_cross_attns = (False, True, True),
    use_linear_attn = True,
    channels = 3,
    channels_out = 3
)


unet2 = SRUnet256(
    dim = 192,
    dim_mults = (1, 2, 4),
    num_resnet_blocks = (2, 4, 8),
    layer_attns = (False, False, True),
    layer_cross_attns = (False, False, True),
    channels = 3,
    channels_out = 3
)


# imagen, which contains the unets above (base unet and super resoluting ones)

imagen = Imagen(
    unets = (unet1, unet2),
    condition_on_text = True,
    text_encoder_name = 't5-small',
    image_sizes = (16, 32),
    timesteps = 1000, # how many steps between gausian steps 1000 prob be better
    channels = 3,
).cuda()

trainer = ImagenTrainer(imagen).cuda()

In [ ]:
MODEL_CHECKPOINT = ""
LABEL_EMBEDDINGS = ""
TRAINING_SET = 1 #0 if creating validation set
DATASET_PATH = ""

In [ ]:
trainer.load(MODEL_CHECKPOINT)
label_embeddings = np.load(LABEL_EMBEDDINGS)

checkpoint loaded from /gdrive/My Drive/Unstable Diffusion/Models/CIFAR Models/hybrid_75-25_gen0_GEN0_2.pt


In [ ]:
gen_val_set = np.zeros((10000, 3, 32, 32))
gen_train_set = np.zeros((50000, 3, 32, 32))
for j in range(max(TRAINING_SET * 5, 1)):
  for label_idx in range(len(label_embeddings)):
    labels = []
    for i in range(1):
      labels += [label_idx] * 1000
    print(labels)
    training_embeddings = label_embeddings[labels]
    training_embeddings = torch.tensor(training_embeddings, requires_grad = True).float().cuda()
    samp = trainer.sample(batch_size = 1, text_embeds = training_embeddings, stop_at_unet_number = 2) 
    gen_val_set[label_idx*1000:label_idx*1000+1000] = samp.cpu().numpy()
  if TRAINING_SET:
    gen_train_set[j*10000: j*10000 + 10000, :, :, :] = np.copy(gen_val_set[:, :, :, :])

In [ ]:
if TRAINING_SET:
  np.save(DATASET_PATH, gen_train_set)
else:
  np.save(DATASET_PATH, gen_val_set)